# Statistical model comparison

In this notebook you will compare two models A and B. We will check that model A is better than B, or not.

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import numpy.testing as np_testing
import matplotlib.pyplot as plt

# Load MAGIC Data Set

<center><img src="img/magic1.png" width="1000"></center>

Source: https://magic.mpp.mpg.de/

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/magic/magic04.data

Features description:
- **Length:** continuous # major axis of ellipse [mm]
- **Width:** continuous # minor axis of ellipse [mm]
- **Size:** continuous # 10-log of sum of content of all pixels [in #phot]
- **Conc:** continuous # ratio of sum of two highest pixels over fSize [ratio]
- **Conc1:** continuous # ratio of highest pixel over fSize [ratio]
- **Asym:** continuous # distance from highest pixel to center, projected onto major axis [mm]
- **M3Long:** continuous # 3rd root of third moment along major axis [mm]
- **M3Trans:** continuous # 3rd root of third moment along minor axis [mm]
- **Alpha:** continuous # angle of major axis with vector to origin [deg]
- **Dist:** continuous # distance from origin to center of ellipse [mm]
- **Label:** g,h # gamma (signal), hadron (background)

g = gamma (signal): 12332 \
h = hadron (background): 6688

In [ ]:
f_names = np.array(["Length", "Width", "Size", "Conc", "Conc1", "Asym", "M3Long", "M3Trans", "Alpha", "Dist"])

data = pd.read_csv("magic04.data", header=None, names=list(f_names)+["Label"])
data.head()

# Data preparation

In [ ]:
# prepare a matrix of input features
X = data[f_names].values

# prepare a vector of true labels
y = 1 * (data['Label'].values == "g")

# take only 10% of the data for speed up
from sklearn.model_selection import train_test_split
X, _, y, _ = train_test_split(X, y, train_size=0.1, stratify=y, random_state=11)

In [ ]:
# print sizes of X and y
X.shape, y.shape

In [ ]:
X[:2]

In [ ]:
y[:5]

# Preprocessing

Scale input data using StandardScaler:
$$
X_{new} = \frac{X - \mu}{\sigma}
$$

In [ ]:
# Import StandardScaler
from sklearn.preprocessing import StandardScaler

# Create object of the class and set up its parameters
ss = StandardScaler()

# Estimate mean and sigma values
ss.fit(X)

# Scale the sample
X = ss.transform(X)

# Define models A and B


We will compare two models: A and B. Model A will have 12 hidden neurons, and model B - only 10. We would like to check that A is better than B.

In [ ]:
#!pip install pytorch-lightning 

In [ ]:
import torch
from torch.nn import functional as F
from torch import nn
import pytorch_lightning as pl

class Model(pl.LightningModule):

    def __init__(self, n_neurons=10):
        super().__init__()
        
        # define all layers of the netwrok
        self.net = nn.Sequential(
                                nn.Linear(10, n_neurons), 
                                nn.Tanh(), 
                                nn.Linear(n_neurons, 1), 
                                nn.Sigmoid())

    
    def forward(self, x):
        # make a prediction for x
        return self.net(x)

    # calculate loss function values
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.binary_cross_entropy(y_hat, y)
        return loss

    # define optimizer to fit the network
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.02)

In [ ]:
model_A = Model(n_neurons=12)
model_A

In [ ]:
model_B = Model(n_neurons=10)
model_B

# Data loader creation

We will define a helping function for converting `X_train` and `y_train` into PyTorch `DataLoader`.

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

def create_data_loader(X_train, y_train, batch_size=128):
    # combine X and y into one pytorch tensor dataset
    dataset_train = TensorDataset(torch.tensor(X_train, dtype=torch.float), 
                                  torch.tensor(y_train.reshape(-1, 1), dtype=torch.float))
    # loader divides our train data into batches
    train_loader = DataLoader(dataset_train, batch_size=batch_size, num_workers=4)
    return train_loader

In [ ]:
# example of usage
create_data_loader(X[:5], y[:5], 1)

# Quality metrics

We will use ROC AUC as a target quality metrics for our models.

In [ ]:
from sklearn import metrics

# example of usage
metrics.roc_auc_score(y_true=[0, 0, 1, 1], 
                      y_score=[0.1, 0.6, 0.8, 0.9])

# Fit models


**Algorithm:**
- Given two models A and B
- Given sample X, y
- For i=1, …, k do:
    - Randomly split $X$ and $y$ into train and test samples
    - Fit models A and B on the train sample
    - Compute quality metrics $q_{Ai}$ and $q_{Bi}$ for the models on the test sample
    - Calculate the difference $q_i = q_{Ai} - q_{Bi}$

Result: we have *k* quality metric differences $\{q_1, q_2, ..., q_k \}$

# Task 1
Using the algorithm above fit models A and B, calculate ROC AUCs for them and estimate the differences q. 

**Hint:** use `n_neurons=10` for model B. Use `model(torch.tensor(X_test, dtype=torch.float))[:, 0].detach().numpy()` to make predictions for a model.

In [ ]:
n_iterations = 50
    
q_A = []
q_B = []

# go through each iteration of KFold
for it in range(n_iterations):

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, stratify=y)
    
    # create pytorch dataloader
    train_loader = create_data_loader(X_train, y_train, batch_size=128)
    
    # init model and trainer for A
    model_A = Model(n_neurons=12)
    trainer_A = pl.Trainer(max_epochs=10, weights_summary=None, progress_bar_refresh_rate=0)
    
    ### BEGIN SOLUTION
    
    ### END SOLUTION
    
    q_A.append(auc_A)
    q_B.append(auc_B)
    
q = np.array(q_A) - np.array(q_B)

In [ ]:
print("Mean value of q: ", q.mean())

Expected approximate output:

<center>   
    
```python
Mean value of q: 0.00312
``` 
    
</center>

In [ ]:
### BEGIN HIDDEN TESTS
actual  = np.array([q.mean()])
desired = np.array([0.003])
np_testing.assert_allclose(actual, desired, atol=0.001)
### END HIDDEN TESTS

In [ ]:
q

# Hypotheses test

We want to test the following hypotheses:
$$
H_0: A = B
$$

$$
H_1: A \text{ is better than } B
$$

We will test them using the bootstrap method.

<center><img src="img/bootstrap.png" width="600"></center>

**Algorithm:**
- For j=1, …, m do:
    - (Bootstrap) Sample with **replacement** a subsample with k objects from $\{q_1, q_2, ..., q_k \}$
    - Calculate mean value $\bar{q}_j$ for the bootstrap sample
- Result: we have m means of the differences $\{\bar{q}_1, \bar{q}_2, ..., \bar{q}_m \}$
- Estimate the probability P that model A is better than model B. This equals to the probability $P$ that $\bar{q}_i > 0$:

# Task 2
Using the algorithm above to estimate means of the differences $\{\bar{q}_1, \bar{q}_2, ..., \bar{q}_m \}$. 

**Hint:** use `resample(q, replace=True, n_samples=len(q))` to make bootstrap samples.

In [ ]:
from sklearn.utils import resample

n_iterations = 1000

q_means = []

# go through each iteration of KFold
for it in range(n_iterations):
    
    ### BEGIN SOLUTION
    
    ### END HIDDEN TESTS
    
q_means = np.array(q_means)

In [ ]:
print("Mean value of q_mean: ", q_means.mean())

Expected approximate output:

<center>   
    
```python
Mean value of q_mean: 0.00312
``` 
    
</center>

In [ ]:
### BEGIN HIDDEN TESTS
P = len(q_means[q_means > 0]) / len(q_means)
actual  = np.array([P])
desired = np.array([0.997])
np_testing.assert_allclose(actual, desired, atol=0.05)
### END HIDDEN TESTS

# Probability that model A is better than model B

$$
P=\frac{1}{k} \sum_{i=1}^{k}I[\bar{q}_i >0]
$$

In [ ]:
plt.hist(q_means, bins=50)
plt.xlabel("q_means", size=14)
plt.show()

In [ ]:
P = len(q_means[q_means > 0]) / len(q_means)

print("Probability that model A is better than model B: ", P)

In [ ]:
alpha = 0.05

if P > 1 - alpha:
    print("A is better than B with significance level ", alpha)
else:
    print("We can not say that A is better than B")